# Task 2.2

In [1]:
!pip install gurobipy

In [3]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = r"C:\Users\Perpendicooler\poland_50_cities_pairwise_distances.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Display the DataFrame
print(df)

                     From          To    Distance
0                 Jakubów      Cegłów    8.884430
1     Rejon ulicy Saperów   Swojczyce    9.315537
2               Paprotnia     Korczew   11.504595
3                  Żurowa   Siedliska   13.365827
4               Węgorzewo     Srokowo   13.940175
...                   ...         ...         ...
1220               Człopa  Hrubieszów  589.668344
1221             Pruchnik   Biały Bór  590.189660
1222               Ostrów   Biały Bór  597.378340
1223             Pruchnik      Cewice  599.645226
1224               Ostrów      Cewice  604.071037

[1225 rows x 3 columns]


# We wrote this code from (https://www.gurobi.com/jupyter_models/traveling-salesman/) here..

# Lazy Row Generation
**The second approach involves a "lazy row generation" version, where constraints
are added progressively until a valid tour is found. This method is known for
its efficiency in solving large instances.**


**The lazy row generation version was implemented using Gurobi's lazy constraints with inspiration from the Gurobi example Gurobi-example.**


***The following function calculates the distance for 50 each pair of cities. Since
we are solving the symmetric traveling salesman problem, we use combinations
of cities***

***We now write the model for the TSP, by defining decision variables, constraints,
and objective function. Because this is the symmetric traveling salesman problem, we can make it more efficient by setting the object x[j,i] to x[i,j], instead
of a constraint***


In [12]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data from Excel
file_path = r"C:\Users\Perpendicooler\poland_50_cities_pairwise_distances.xlsx"
data = pd.read_excel(file_path)

# Extract city names and distances
cities = set(data["From"].tolist() + data["To"].tolist())
city_index = {city: i for i, city in enumerate(cities)}
distances = {(city_index[row.From], city_index[row.To]): row.Distance for row in data.itertuples()}

# Create a Gurobi model
m = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?
vars = m.addVars(distances.keys(), obj=distances, vtype=GRB.BINARY, name='x')

# Symmetric direction: Copy the object
keys = list(vars.keys())  # Create a list of keys
for i, j in keys:
    vars[j, i] = vars[i, j]  # edge in opposite direction

# Constraints: two edges incident to each city
capitals = [i for i in range(len(cities))]
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

# Optimize the model
m.optimize()

# Print the tour
tour = [i for i, j in vars.keys() if vars[i, j].X > 0.5]
print("Tour:", tour)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 3 4300U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 50 rows, 1225 columns and 2450 nonzeros
Model fingerprint: 0xe0aab684
Variable types: 0 continuous, 1225 integer (1225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 13005.316269
Presolve time: 0.00s
Presolved: 50 rows, 1225 columns, 2450 nonzeros
Variable types: 0 continuous, 1225 integer (1225 binary)

Root relaxation: objective 2.687795e+03, 88 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2687.79520    0   1

**The Gurobi Optimizer is solving a mathematical optimization problem with 50 rows, 1225 columns, and 2450 nonzeros. The model involves both continuous and integer variables. The solver initially found a heuristic solution with an objective value of 13005.316269.**

**After presolving the model, the optimization process explores nodes in a search tree, refining the solution. The final optimal solution is found with an objective value of 2.698101399109e+03, and the solver reports a 0% optimality gap. The sequence of decisions forming the optimal solution is provided as a "tour."**

Adding Lazy Constraints

In [22]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour) - 1)

# Given a tuplelist of edges, find the shortest subtour
def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:]  # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle  # New shortest subtour
    return cycle

# Set the callback function
m._vars = vars
m.Params.LazyConstraints = 1
m.optimize(subtourelim)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: AMD Ryzen 3 4300U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 50 rows, 1225 columns and 2450 nonzeros
Model fingerprint: 0xe0aab684
Variable types: 0 continuous, 1225 integer (1225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolved: 50 rows, 1225 columns, 2450 nonzeros

Continuing optimization...


Cutting planes:
  Zero half: 3

Explored 1 nodes (88 simplex iterations) in 0.07 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 2698.1 2786.02 2810.06 13005.3 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.698101399109e+03, best bound 2.698101399109e+03, gap 0.0000%

User-callback calls 30, time in user-cal

**The model involves both continuous and integer variables. After presolving, the solver continues optimization and applies cutting planes. It explores one node, finding four solutions. The optimal solution is found with an objective value of 2.698101399109e+03 and a 0% optimality gap. Additionally, user-callback functions were called 30 times during the optimization process.**